# YouTube Data Collection with the youtube_data_processing Package

-------------------------------------------------------------

Notebook created by Marcelo, David and GPT, Aug 2023.

-------------------------------------------------------------

Welcome to this tutorial on YouTube data collection! In this notebook, we will explore how to use the youtube_data_processing module, a custom Python module designed to interact with the YouTube Data API, fetch data from it, and organize the data for further analysis.

The youtube_data_processing module includes functionalities to fetch statistics about YouTube channels, retrieve a list of videos uploaded by a given channel, and gather detailed information about each video. It also provides utility functions to save the fetched data into JSON files and read data from them. This efficient data collection and storage approach makes it easier to manage large amounts of data and control API quota usage.

The module is designed with simplicity and efficiency in mind, allowing you to gather YouTube data with just a few lines of Python code. Whether you are a YouTube channel owner looking to gain insights into your channel's performance, a data scientist wanting to analyze YouTube trends, or a student learning data science and looking for interesting datasets to work with, the youtube_data_processing module can be a great tool to streamline your data collection process.

In this tutorial, we will cover:

    Setting up the YouTube Data API and getting your API key.
    Fetching data from YouTube using the youtube_data_processing package.
    Understanding the structure of the fetched data.
    Creating flat tables from the fetched data for easier analysis.
    Saving and loading data using JSON files.
    Plus some additional functions

Let's get started and dive into the exciting world of YouTube data!

### Preliminary

First, we need to import the youtube_data_processing module that we'll be using in this tutorial. We can do this with the following line of code:

In [16]:
import pandas as pd
import glob
import json

In [3]:
# In case you need to work updte the modeule. This imports the importlib module, 
# which contains functions that help you control 
# the runtime process of Python scripts, especially those related to importing and reloading modules.
import importlib

# Then imports the youtube_data_processing module under the alias yt.
from packages import youtube_data_processing as yt

# This reloads the youtube_data module. The purpose of this is to ensure that the 
# latest version of the module is in use, especially if the module has been modified 
# since the start of the Python session.
importlib.reload(yt)

<module 'packages.youtube_data_processing' from '/home/mamaral/Documents/qgr/codes/python/notebooks/computational_essays/packages/youtube_data_processing.py'>

The youtube_data_processing module interacts with the YouTube Data API, and thus requires an API key. If you don't already have an API key, you'll need to set one up. Here's how you can do that:

    Log in to your YouTube account.
    Navigate to the Google Developers Console (https://console.developers.google.com/).
    Create a new project in the console.
    In your new project dashboard, click "Explore & Enable APIs".
    In the library, find the "YouTube Data API v3" under "YouTube APIs" and enable it.
    After enabling the API, you'll need to create a credential for it. This is what will give you your API key.

Once you've done all this, you'll be presented with an API key that you can use in this tutorial. Remember to keep this key safe and don't share it publicly. For more detailed information about getting started with the YouTube Data API, you can check out Google's Getting Started guide (https://developers.google.com/youtube/v3/getting-started).

### Overview of the functions

This script essentially collects data from YouTube using the YouTube Data API, processes the data, and organizes it into two dataframes: one for channel data and another for video data. The data collected includes various attributes of channels and videos, such as channel description, video count, view count, subscriber count, video title, publish date, like count, comment count, and many more.

Here's a summary of what each function does:

    get_channel_statistics(api_key, channel_id): Fetches the statistics for a given YouTube channel using the YouTube Data API. The statistics include the number of subscribers, total views, and total videos, among other data.

    write_channel_video_list_to_file(channel_id, video_ids): Writes a list of video IDs to a JSON file. Each video ID is associated with a flag indicating whether the video's data has been fetched from the API.

    get_channel_videos_list(channel_id, api_key): Retrieves a list of video IDs for the videos uploaded by a given YouTube channel. This is done using the YouTube Data API's search functionality, which returns a list of videos in reverse chronological order (i.e., newest videos first).

    get_channel_video(api_key, video_id): Retrieves detailed data for a single video using the YouTube Data API. The data includes the video's title, description, duration, view count, like count, comment count, and more.

    process_channels(api_key, max_channels, api_data_file, channels_file_json): Processes a list of YouTube channels, fetching their statistics and videos using the YouTube Data API. The data is written to a JSON file. The function also manages a quota limit by keeping track of the number of channels processed and stopping when a maximum limit is reached.

    create_flat_tables_youtube_data(df_api, df_trend, df_category): Creates two flat tables for the YouTube data. The function uses the API data, trending data, and category data to create two tables: one for the channel data and another for the video data. The function also updates the tables with information on whether a channel has a trending video and the number of trending videos a channel has. The tables are then written to .xlsx files.

    fetch_youtube_data(api_key, api_data_file, channels_file_json, maxChannels): This function orchestrates the entire process of fetching data from the YouTube API. It first checks whether there are pre-existing data files; if there are, it loads them; if not, it creates new ones. The function then iterates through a list of channel IDs. For each channel, it checks whether the channel's data has already been fetched. If not, it fetches the data and updates the data files. The function does the same for each video of the channel.

The script also handles two types of JSON data:

    A list of channels to process, where each channel has a channel ID, a flag indicating whether its statistics have been fetched (fetched_statistics), and a flag indicating whether its videos have been fetched (fetched_videos).
    A list of video IDs for each channel, where each video ID is associated with a flag indicating whether the video's data has been fetched (fetched_video).

### Fetching Data

In this section, we will be using the fetch_youtube_data function to retrieve data from the YouTube Data API. The function takes four parameters:

    api_key: This is your YouTube API key, which authenticates you to the API and allows you to make data requests.

    api_data_file: This is the JSON file where the fetched data will be stored. The file will consist of a dictionary, where each key-value pair corresponds to a channelId and its associated data. The dictionary's 'statistics' field contains the statistics for the channel itself, such as the number of subscribers, total views, and total videos, among other data. The 'videos' field contains a list of videos uploaded by the channel. Each entry in this list is a dictionary containing data about a single video, including the video's title, description, view count, like count, comment count, and other details.

    channels_file_json: This is the JSON file containing a list of channels to process. Each channel in the list has a channel ID, a flag indicating whether its statistics have been fetched (fetched_statistics), and a flag indicating whether its videos have been fetched (fetched_videos).

    maxChannels: This is the maximum number of channels to process. The function will stop processing once it has processed the specified number of channels.

In the code snippet below, we're setting the api_key to your actual YouTube API key, the api_data_file to 'youtube_selected_data_API_US.json', the channels_file_json to 'our_YT_channel_ids.json', and maxChannels to 3. We then call the fetch_youtube_data function to begin the data retrieval process.

In [8]:
# api_data_file should have dictionary where each key-value pair corresponds to a channelId and its associated data.
#     statistics: This field contains the statistics for the channel itself, 
# likely including things like the number of subscribers, the total number of views, and the total number of videos, 
# among other data.  {'yt_api': {'videos': [], 'statistics':...
#    videos: This field contains a list of videos uploaded by the channel. 
# Each entry in this list is a dictionary containing data about a single video, 
# which might include the video's title, description, view count, like count, comment count, and other details.
api_data_file = "data/YT_data_sample.json"

# channels_file_json has the structure: 
# [{"channelID": "UCdWIQh9DGG6uhJk8eyIFl1w", "fetched_videos": true, "fetched_statistics": true},{...}..]
channels_file_json = "data/YT_channel_ids.json"

# Your API key
api_key = "your_api_key" 

# Set a maximum number of channels to process
maxChannels = 1

# retrieve data from the YouTube Data API
yt.fetch_youtube_data(api_key, api_data_file, channels_file_json, maxChannels)


1
1


In [6]:
# Updates:
# - Loop thru a set of api keys
# - Automatically name file

#channel_file_json = "yt_trending_channelid_selection_ranked_updated.json"
channel_file_json = "data/YT_channel_ids.json"
maxChannels = 100
api_keys = ['key1', 'key2']

base_number = 1 

for api in api_keys:
    api_video_file = "data/youtube_data_API_US_" + str(base_number) + "_" + str(base_number+40) + ".json"
    #print(api_video_file)
    try:
        print(api)
        yt.fetch_youtube_data(api, api_video_file, channel_file_json, maxChannels, True)
    except Exception as e: # Catch the specific exception and print it
        print(f"API Key used up for the day: {api}")
        print(f"Error details: {str(e)}")
print(api_video_file)

key1
No videos found for channel UCd_hVSCsmp9viJpr41765dA
API Key used up for the day: key1
Error details: 'NoneType' object is not iterable
key2
No videos found for channel UCd_hVSCsmp9viJpr41765dA
API Key used up for the day: key2
Error details: 'NoneType' object is not iterable
data/youtube_data_API_US_1_41.json


### Creating flat tables from the fetched data for easier analysis

In this section, we leverage the data we fetched from the YouTube Data API to create structured tables that will facilitate further analysis. We accomplish this by reading our JSON files into pandas DataFrames and then processing them with our custom function create_flat_tables_youtube_data from the youtube_data_processing package.

This function creates two tables: one for channel data and another for video data. It enriches the channel data with information on whether a channel has a trending video and the number of trending videos a channel has. The resulting tables are saved as .xlsx files, providing a structured and convenient format for any subsequent analysis. This section illustrates the transformation of raw, unstructured data into organized, structured data that is ready for detailed examination.

In [9]:
# This line uses the pandas function read_json to read a JSON file and convert it 
# into a pandas DataFrame. The file 'youtube_selected_data_API_US_54.json' presumably 
# contains data fetched from the YouTube Data API. The 'lines=True' argument is 
# used because the file is in a JSON Lines format, where each line is a valid JSON object.
# youtube_selected_data_API_US_54.json should have dictionary where each key-value pair corresponds to a channelId 
# and its associated data. {'yt_api': {'videos': [], 'statistics':...
#     statistics: This field contains the statistics for the channel itself, 
# likely including things like the number of subscribers, the total number of views, and the total number of videos, 
# among other data.
#    videos: This field contains a list of videos uploaded by the channel. 

df_api = pd.read_json('data/YT_data_sample.json', lines=True)


# This line is similar to the previous one, but it reads the file 
# 'youtube_300_selected_data_trend_US.json', which presumably contains data on 
# trending videos fetched from a trending database.
# the structure is videoID  \
# 0  {'US_trends': [{'row_id': 1424, 'video_id': ....' ...
df_trend = pd.read_json('data/sampled_youtube_data_trend.json', lines=True)

# This block opens the file 'US_category_id.json' and loads the JSON data into the 
# variable df_category. This file presumably contains data on the different video 
# categories on YouTube.
with open('data/US_category_id.json') as f:
    df_category = json.load(f)


xlsx_channel_name = 'data/YT_data_sample_channel.xlsx'
xlsx_channel_video = 'data/YT_data_sample_video.xlsx'

# Finally, this line calls the create_flat_tables_youtube_data function from the 
# youtube_data module, passing the three dataframes as arguments. This function 
# creates two flat tables for the YouTube data (one for the channel data and one 
# for the video data) and writes them to .xlsx files.
yt.create_flat_tables_youtube_data(df_api, df_trend, df_category, xlsx_channel_name, xlsx_channel_video)


Empty video data encountered.
Empty video data encountered.
Empty video data encountered.
Empty video data encountered.


In this section, we load the structured data from the .xlsx files we created in the previous step. The data is read into pandas DataFrames, which are convenient structures for handling and analyzing tabular data in Python.

We have two DataFrames: channel_table and video_table, corresponding to the channel and video data, respectively. By using the head() function, we can display the first few rows of each DataFrame. This helps provide an initial overview of the data structure and the information contained within.

This step is crucial in familiarizing ourselves with the data before proceeding with any further analysis or processing.

In [11]:
# Load the channel and video data from the xlsx files
channel_table = pd.read_excel('data/YT_data_sample_channel.xlsx')
video_table = pd.read_excel('data/YT_data_sample_video.xlsx')

print(channel_table.columns.tolist())
print(video_table.columns.tolist())

# Display the first few rows of each dataframe
print(channel_table.head())
print(video_table.head())



['channelId', 'channelTitle', 'description', 'publishedAt', 'videoCount', 'viewCount', 'subscriberCount', 'country', 'customUrl', 'topicCategories', 'madeForKids', 'keywords', 'hasVideoTrending', 'numberVideoTrending']
['videoId', 'title', 'description', 'publishedAt', 'channelId', 'categoryId', 'category', 'viewCount', 'likeCount', 'commentCount', 'tags', 'channelTitle', 'thumbnails', 'isTrending', 'duration', 'dimension', 'definition', 'caption', 'licensedContent', 'projection', 'uploadStatus', 'privacyStatus', 'license', 'embeddable', 'publicStatsViewable', 'madeForKids', 'favoriteCount', 'topicCategories']
                  channelId              channelTitle  \
0  UCUyk0KLo7JPLCCh4oRNLzsQ  Quantum Gravity Research   

                                         description           publishedAt  \
0  Quantum Gravity Research is a Los Angeles base...  2015-11-17T00:34:43Z   

   videoCount  viewCount  subscriberCount country                customUrl  \
0         186   11257198        

### Eliminate duplicated

In [13]:

# Load the Excel file into a DataFrame
channel_table = pd.read_excel('data/YT_data_sample_channel.xlsx')
print("Total number of rows:", channel_table.shape[0])

# Remove duplicates based on the 'channelId' column and keep the first occurrence
channel_table_cleaned = channel_table.drop_duplicates(subset='channelId', keep='first')
print("Total number of rows:", channel_table_cleaned.shape[0])

# Save the cleaned DataFrame back to the Excel file
#channel_table_cleaned.to_excel('data/YT_data_sample_video_clean.xlsx', index=False)

# Load the Excel file into a DataFrame
video_table = pd.read_excel('data/YT_data_sample_video.xlsx')
print("Total number of rows:", video_table.shape[0])

# Remove duplicates based on the 'videoId' column and keep the first occurrence
video_table_cleaned = video_table.drop_duplicates(subset='videoId', keep='first')
print("Total number of rows:", video_table_cleaned.shape[0])

# Save the cleaned DataFrame back to the Excel file
#video_table_cleaned.to_excel('data/YT_data_sample_video_clean.xlsx', index=False)


Total number of rows: 1
Total number of rows: 1
Total number of rows: 182
Total number of rows: 181


### Data enrichement

This function enriches YouTube channel and video data with calculated statistics.

For each channel, the function calculates the mean and standard deviation of 'viewCount', 'likeCount',
'commentCount', and 'favoriteCount'. It then merges this statistical data with the original channel data.

For the video data, the function calculates the ratio of each video's 'viewCount', 'likeCount', 'commentCount',
and 'favoriteCount' to the average count for the respective channel.

The enriched data is saved back into new .xlsx files.

Parameters:
xlsx_channels (str): The path to the .xlsx file containing the channel data.
xlsx_videos (str): The path to the .xlsx file containing the video data.
xlsx_channels_enriched (str): The path to the .xlsx file to write the enriched channel data.
xlsx_videos_enriched (str): The path to the .xlsx file to write the enriched video data.

In [14]:
xlsx_channels = 'data/YT_data_sample_channel.xlsx'
xlsx_videos = 'data/YT_data_sample_video.xlsx'
xlsx_channels_enriched = 'data/YT_data_sample_channel_enriched.xlsx'
xlsx_videos_enriched = 'data/YT_data_sample_video_enriched.xlsx'

yt.enriche_xlsx_youtube_data(xlsx_channels, xlsx_videos, xlsx_channels_enriched, xlsx_videos_enriched)


### Merge tables

We can have different tables, so we can merge them if we want.

In [15]:
# merge channel tables
#xlsx_file1 = "data/YT_data_sample_channel_enriched.xlsx"
#xlsx_file2 = "data/YT_data_sample_channel_enriched2.xlsx"
#xlsx_output_filename = "data/YT_data_sample_channel_enriched_merged.xlsx"
#yt.merge_xlsx_files(xlsx_file1, xlsx_file2, xlsx_output_filename)

# merge videos tables
#xlsx_file1 = "data/YT_data_sample_video_enriched.xlsx"
#xlsx_file2 = "data/YT_data_sample_video_enriched2.xlsx"
#xlsx_output_filename = "data/YT_data_sample_video_enriched_merged.xlsx"
#yt.merge_xlsx_files(xlsx_file1, xlsx_file2, xlsx_output_filename)


In [23]:
#full folder

# Path to the folder containing the channel files
channel_files_path = 'data/xlsx/channel_table_*.xlsx'

# Read the updated merged data
data_ini_path = 'data/xlsx/YT_data_sample_channel_enriched_merged.xlsx'
merged_data = pd.read_excel(data_ini_path)

# List of channel files
channel_files = glob.glob(channel_files_path)

# Loop through channel files
for channel_file in channel_files:
    if channel_file == data_ini_path:
        continue
    channel_data = pd.read_excel(channel_file)
    print(channel_file)
    # Concatenate the two dataframes into one
    merged_data = pd.concat([merged_data, channel_data], ignore_index=True)

# Save the final merged data
output_filename = 'YT_data_sample_channel_enriched_merged_full.xlsx'
merged_data.to_excel(output_filename, index=False)

xlsx/channel_table_youtube_patreon123_153_data_enriched.xlsx
xlsx/channel_table_youtube_patreon80_123_data_enriched.xlsx
xlsx/channel_table_youtube_trend175_200_data_enriched.xlsx
xlsx/channel_table_youtube_trend125_150_data_enriched.xlsx
xlsx/channel_table_youtube_trend0_25_data_enriched.xlsx
xlsx/channel_table_youtube_trend200_230_data_enriched.xlsx
xlsx/channel_table_youtube_trend150_175_data_enriched.xlsx
xlsx/channel_table_youtube_trend75_100_data_enriched.xlsx
xlsx/channel_table_youtube_trend25_50_data_enriched.xlsx
xlsx/channel_table_youtube_trend100_125_data_enriched.xlsx
xlsx/channel_table_youtube_patreon39_80_data_enriched.xlsx
xlsx/channel_table_youtube_patreon1_39_data_enriched.xlsx
xlsx/channel_table_youtube_trend50_75_data_enriched.xlsx


In [24]:
# Path to the folder containing the video files
video_files_path = 'xlsx/video_table_*.xlsx'

# Read the updated merged data
data_ini_path = 'xlsx/YT_data_sample_video_enriched_merged.xlsx'
merged_data = pd.read_excel(data_ini_path)

# List of video files
video_files = glob.glob(video_files_path)

# Loop through video files
for video_file in video_files:
    if video_file == data_ini_path:
        continue
    video_data = pd.read_excel(video_file)
    print(video_file)
    # Concatenate the two dataframes into one
    merged_data = pd.concat([merged_data, video_data], ignore_index=True)

# Save the final merged data
output_filename = 'YT_data_sample_video_enriched_merged_full.xlsx'
merged_data.to_excel(output_filename, index=False)

xlsx/video_table_youtube_trend200_230_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_patreon39_80_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_patreon1_39_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_trend0_25_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_trend150_175_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_trend175_200_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_trend75_100_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_trend25_50_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_trend50_75_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_patreon80_123_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_patreon123_153_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_trend100_125_data_cleaned_enriched.xlsx
xlsx/video_table_youtube_trend125_150_data_cleaned_enriched.xlsx


### Eliminate/check duplicated videos in .json

In [115]:
# Read the JSON data into a DataFrame
df = pd.read_json('yt_trending_channelid_selection_ranked.json')

# Count the channels where fetched_videos is True
processed_channels_count = df[df['fetched_videos'] == True].shape[0]

# Print the result
print(f"Number of channels processed: {processed_channels_count}")


Number of channels processed: 201


In [116]:
# Filter the DataFrame to include only the processed channels (fetched_videos = True)
processed_channels_df = df[df['fetched_videos'] == True]

# Check for duplicates in the 'channelID' column and sum the result to get the count of duplicates
duplicated_channels_count = processed_channels_df['channelID'].duplicated().sum()

# Print the result
print(f"Number of duplicated channels among processed ones: {duplicated_channels_count}")


Number of duplicated channels among processed ones: 0


In [99]:
# Read the channel IDs from the text file
with open('channelsProcessed', 'r') as file:
    processed_channels = set(line.strip() for line in file)

# Load the JSON data from the file
with open('yt_trending_channelid_selection_ranked.json', 'r') as file:
    data = json.load(file)

# Update the values based on whether the channel ID is in the set
for item in data:
    channel_id = item['channelID']
    item['fetched_videos'] = channel_id in processed_channels
    item['fetched_statistics'] = item['fetched_videos']

# Sort the data so that processed ones come first
data.sort(key=lambda x: x['fetched_videos'], reverse=True)

# Save the updated data back to a JSON file
with open('yt_trending_channelid_selection_ranked_updated.json', 'w') as file:
    json.dump(data, file)


In [100]:
# Read the JSON data into a DataFrame
df = pd.read_json('yt_trending_channelid_selection_ranked_updated.json')

# Count the channels where fetched_videos is True
processed_channels_count = df[df['fetched_videos'] == True].shape[0]

# Print the result
print(f"Number of channels processed: {processed_channels_count}")

Number of channels processed: 199


### Additional Functions

In [68]:
# create a new json file to process with fetch_youtube_data from csv with channel ids
channels_id_csv_file = "yt_channel_aparently_not_processed.csv"
channels_id_json_file = "yt_channel_aparently_not_processed.json"
yt.channel_ids_csv_to_json(channels_id_csv_file,channels_id_json_file)

File saved: yt_channel_aparently_not_processed.json.


In [17]:
# get the top videos by channel

xlsx_videos = 'data/YT_data_sample_video_enriched.xlsx'

top_videos = yt.get_top_videos(xlsx_videos, 2)

for channelId, group in top_videos.groupby('channelId'):
    print(f"Channel: {channelId}, Channel Title: {group.iloc[0]['channelTitle']}")
    for index, row in group.iterrows():
        print(f"  Video: {row['title']}, Views: {row['viewCount']}")
    print()

Channel: UCUyk0KLo7JPLCCh4oRNLzsQ, Channel Title: Quantum Gravity Research
  Video: What Is Reality? [Official Film], Views: 4244127
  Video: Hacking Reality [Official Film], Views: 2052430

